# Script for Creating SQL Database

In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

## DB and Table Creation
From DnD excel file

In [2]:
# If building db from scratch, run this cell
import os
db_name = "lldm.db"
if os.path.exists(db_name):
    os.remove(db_name)

In [3]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print("Database lldm.db formed.") 
except: 
    print("Database lldm.db not formed.")

Database lldm.db formed.


In [6]:
dfs = pd.read_excel('DnD.xlsx', sheet_name=None)
for table, df in dfs.items():
    try:
        print(table.upper().replace(' ','_').strip())
        df.columns = [x.replace(' ','_') for x in df.columns.values]
        df.to_sql(table.upper().replace(' ','_').strip(), db)
    except:
        continue

CHARACTER_SHEET
INVENTORY
WORLD_ITEMS
SETTINGS
NPCS
TREASURES
MONSTERS
PLOT
LOGS


In [9]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

In [10]:
# add campaign table
query = '''CREATE TABLE IF NOT EXISTS CAMPAIGN (
	Campaign_ID INTEGER NOT NULL,
   	Setting TEXT NOT NULL,
    Start_Time TEXT NOT NULL,
    Current_Turns INTEGER NOT NULL DEFAULT 0
);
'''

run_query(db, query)

In [11]:
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
cursor = db.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('CHARACTER_SHEET',), ('INVENTORY',), ('WORLD_ITEMS',), ('SETTINGS',), ('NPCS',), ('TREASURES',), ('MONSTERS',), ('PLOT',), ('LOGS',), ('CAMPAIGN',)]


## Data Check

In [12]:
query = '''
SELECT * FROM CHARACTER_SHEET
'''
run_query(db, query)

,index,Name,Pronouns,Race,Class,Alignment,Brief_Description,Appearance,Backstory,Personality,Strength,Intelligence,Wisdom,Constitution,Dexterity,Charisma
0,0,Elara Windrider,She/Her,Human,Fighter,Lawful Good,Courageous warrior with a heart of gold,"Tall and athletic, with short brown hair, green eyes, and a determined expression. Wears chain mail and carries a longsword.","Elara was born in a small village and trained by her father, a retired soldier. She left home to protect the innocent and seek justice.","Brave and compassionate, with a strong sense of justice. She is determined and reliable, but can be stubborn at times.",14,10,11,13,12,14


In [13]:
query = '''
SELECT * FROM INVENTORY
'''
run_query(db, query)

,index,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility purposes.
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobility.
9,9,Armor,Shield,Additional defense against attacks.


In [17]:
query = '''
SELECT * FROM WORLD_ITEMS
'''
run_query(db, query)

,Item_ID,Weapon_Name,Weapon_Description,Prompt
0,0,Legendary Wand of Assassin,"The Legendary Wand of Assassin is a wand known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this wand is imbued with the essence of mage. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Legendary Wand of Assassin in detail, including its history, special abilities, and appearance."
1,1,Forgotten Lance of Knight,"The Forgotten Lance of Knight is a lance known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this lance is imbued with the essence of dragon. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Forgotten Lance of Knight in detail, including its history, special abilities, and appearance."
2,2,Cursed Armor of Warrior,"The Cursed Armor of Warrior is a armor known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this armor is imbued with the essence of ranger. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Cursed Armor of Warrior in detail, including its history, special abilities, and appearance."
3,3,Divine Bow of Ranger,"The Divine Bow of Ranger is a bow known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this bow is imbued with the essence of mage. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Divine Bow of Ranger in detail, including its history, special abilities, and appearance."
4,4,Cursed Armor of Storm,"The Cursed Armor of Storm is a armor known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this armor is imbued with the essence of frost. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Cursed Armor of Storm in detail, including its history, special abilities, and appearance."
...,...,...,...,...
95,95,Glowing Armor of Warrior,"The Glowing Armor of Warrior is a armor known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this armor is imbued with the essence of warrior. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Glowing Armor of Warrior in detail, including its history, special abilities, and appearance."
96,96,Divine Ring of Mage,"The Divine Ring of Mage is a ring known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this ring is imbued with the essence of frost. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Divine Ring of Mage in detail, including its history, special abilities, and appearance."
97,97,Shadow Lance of Storm,"The Shadow Lance of Storm is a lance known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this lance is imbued with the essence of mage. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Shadow Lance of Storm in detail, including its history, special abilities, and appearance."
98,98,Forgotten Sword of Flame,"The Forgotten Sword of Flame is a sword known throughout the realms for its incredible power. Forged in the fires of an ancient forge, this sword is imbued with the essence of flame. It is said that those who wield it gain unmatched abilities, making them formidable in any battle.","Describe the weapon named Forgotten Sword of Flame in detail, including its history, special abilities, and appearance."


In [14]:
query = '''
SELECT * FROM SETTINGS
'''
run_query(db, query)

,index,Setting,Name,Description,Key_Events,X_Coordinate,Y_Coordinate
0,0,Traven,The Dragon's Flagon,"A bustling tavern located in the heart of the town of Windshade. The tavern is filled with adventurers, merchants, and townsfolk.",Elara hears a rumor about a powerful sword hidden in the Whispering Woods and decides to embark on a quest to find it.,155,155
1,1,Wilderness,Whispering Woods,A dense and mysterious forest located to the north of Windshade. The woods are known for their eerie silence and are filled with ancient trees.,"Elara navigates through the forest, facing various non-combat challenges like tricky terrain, ancient traps, and solving puzzles to find the hidden sword. After overcoming these challenges, she discovers the enchanted sword in a small, overgrown shrine deep within the woods.",220,250
2,2,Castle,Blackstone Keep,"An ancient and abandoned castle perched on a hilltop, surrounded by dark, ominous clouds. It is rumored to be the lair of a fearsome monster.","Armed with the sword, Elara enters Blackstone Keep, faces the fearsome dragon, and engages in a climactic battle, ultimately defeating it.",50,280


## Modify Tables
Add primary keys and rename bad columns

### Character Sheet

In [33]:
query = '''
ALTER TABLE CHARACTER_SHEET
RENAME COLUMN "index" TO Character_ID;

'''
run_query(db, query)

In [34]:
query = '''
ALTER TABLE CHARACTER_SHEET
ADD COLUMN Campaign_ID INTEGER;
'''
run_query(db, query)

In [35]:
query = '''
SELECT * FROM CHARACTER_SHEET
'''
run_query(db, query)

,Character_ID,Name,Pronouns,Race,Class,Alignment,Brief_Description,Appearance,Backstory,Personality,Strength,Intelligence,Wisdom,Constitution,Dexterity,Charisma,Campaign_ID
0,0,Elara Windrider,She/Her,Human,Fighter,Lawful Good,Courageous warrior with a heart of gold,"Tall and athletic, with short brown hair, green eyes, and a determined expression. Wears chain mail and carries a longsword.","Elara was born in a small village and trained by her father, a retired soldier. She left home to protect the innocent and seek justice.","Brave and compassionate, with a strong sense of justice. She is determined and reliable, but can be stubborn at times.",14,10,11,13,12,14,None


### Inventory
Inventory is essentially a master reference table of all possible items that can be used in a campaign - campaign agnostic. Shouldn't be altered unless functionality to add new items in universe is implemented

In [36]:
query = '''
ALTER TABLE INVENTORY
RENAME COLUMN "index" TO Item_ID;
'''
run_query(db, query)

In [37]:
query = '''
SELECT * FROM INVENTORY
'''
run_query(db, query)

,Item_ID,Category,Item,Description
0,0,Weapon,Longsword,Elara's main weapon for combat.
1,1,Weapon,Dagger,Versatile tool for close combat and utility purposes.
2,2,Adventuring Gear,Backpack,To carry essential items.
3,3,Adventuring Gear,Rope (50 feet),Useful for climbing and securing objects.
4,4,Adventuring Gear,Rations (5 days),Enough food for the journey.
5,5,Adventuring Gear,Water skin,To carry water.
6,6,Adventuring Gear,Flint and Steel,For starting fires.
7,7,Adventuring Gear,Healing Potions (2),For quick recovery in emergencies.
8,8,Armor,Chain Mail,Provides solid protection while allowing mobility.
9,9,Armor,Shield,Additional defense against attacks.


### World Items

In [16]:
query = '''
ALTER TABLE WORLD_ITEMS
RENAME COLUMN "index" TO Item_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

### Settings
Reference table for possible campaign settings, campaign agnostic. For now, should only be referenced, not modified during campaigns.

In [18]:
query = '''
ALTER TABLE SETTINGS
RENAME COLUMN "index" TO Setting_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [19]:
query = '''
SELECT * FROM SETTINGS
'''
run_query(db, query)

,Setting_ID,Setting,Name,Description,Key_Events,X_Coordinate,Y_Coordinate
0,0,Traven,The Dragon's Flagon,"A bustling tavern located in the heart of the town of Windshade. The tavern is filled with adventurers, merchants, and townsfolk.",Elara hears a rumor about a powerful sword hidden in the Whispering Woods and decides to embark on a quest to find it.,155,155
1,1,Wilderness,Whispering Woods,A dense and mysterious forest located to the north of Windshade. The woods are known for their eerie silence and are filled with ancient trees.,"Elara navigates through the forest, facing various non-combat challenges like tricky terrain, ancient traps, and solving puzzles to find the hidden sword. After overcoming these challenges, she discovers the enchanted sword in a small, overgrown shrine deep within the woods.",220,250
2,2,Castle,Blackstone Keep,"An ancient and abandoned castle perched on a hilltop, surrounded by dark, ominous clouds. It is rumored to be the lair of a fearsome monster.","Armed with the sword, Elara enters Blackstone Keep, faces the fearsome dragon, and engages in a climactic battle, ultimately defeating it.",50,280


### NPCs
NPC reference table, not to be modified

In [20]:
query = '''
ALTER TABLE NPCS
RENAME COLUMN "index" TO NPC_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [21]:
query = '''
SELECT * FROM NPCS
'''
run_query(db, query)

,NPC_ID


### Treasures
Trasure reference table, not to be modified

In [22]:
query = '''
ALTER TABLE TREASURES
RENAME COLUMN "index" TO Treasure_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [23]:
query = '''
SELECT * FROM TREASURES
'''
run_query(db, query)

,Treasure_ID,Name,Type,Description,Properties,Damage,Special_Abilities,Background
0,0,Sword of the Phoenix,Longsword,"A beautifully crafted longsword with a hilt shaped like a phoenix, glowing with fiery runes.","+1 to hit and damage rolls, additional fire damage, flame burst ability",1d8+1 slashing damage + 1d6 fire damage,"Flame Burst: Once per day, the sword can unleash a burst of flame, dealing 3d6 fire damage to all creatures within a 10-foot radius (save vs. spells for half damage).","Forged in the heart of a volcano by ancient fire giants, the Sword of the Phoenix has been wielded by heroes throughout the ages. It is said to grant its bearer the strength and courage of the mythical phoenix."


### Monsters
Monster reference table, not to be modified

In [24]:
query = '''
ALTER TABLE MONSTERS
RENAME COLUMN "index" TO Monster_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [25]:
query = '''
SELECT * FROM MONSTERS
'''
run_query(db, query)

,Monster_ID,Name,Type,Alignment,Hit_Points,Armor_Class,Move,Strength,Dexterity,Constitution,Intelligence,Wisdom,Charisma,Saving_Throws,Special_Abilities,Damage_Immunities,Senses,Languages,Challenge_Level,Actions,Lair_Actions,Description,Encounter
0,0,Shadowflame,Dragon,Chaotic Evil,64 (8d8+24),2 (descending AC as per 1974 rules),"12"" / 24"" (ground/fly)",Exceptional (Dragons have high strength by default in 1974 D&D),Average,High (Dragons have significant hit points and resilience),Average,Average,High (Dragons have a commanding presence),As 8th-level Fighter (save as per 1974 D&D saving throw tables),"Breath Weapon: Fire, 6"" long and 2"" wide cone, dealing 45 (13d6) fire damage. Save vs. Dragon Breath for half damage.<br>Legendary Resistance: If Shadowflame fails a saving throw, it can choose to succeed instead, up to 3 times per day.",Fire,"Vision: Can see in total darkness up to 6"" (infravision)","Common, Dragon",7,"Multiattack: Shadowflame makes three attacks: one with its bite and two with its claws.<br>Bite: +9 to hit, 2"" reach, one target. Hit: 15 (2d10+6) damage.<br>Claw: +9 to hit, 1"" reach, one target. Hit: 11 (2d6+6) damage.<br>Tail Swipe: +7 to hit, 2"" reach, all creatures within 2"" of the dragon. Hit: 10 (2d6+4) damage.","On initiative count 20 (losing initiative ties), Shadowflame takes a lair action to cause one of the following effects:<br>Volcanic Gas: A cloud of volcanic gas fills a 2"" radius sphere centered on a point Shadowflame can see within 12"" of it. The sphere spreads around corners, and its area is heavily obscured. It lasts until the initiative count 20 on the next round.<br>Lava Burst: A lava burst erupts from a point on the ground Shadowflame can see within 12"" of it. Each creature within 1"" of that point must make a saving throw, taking 21 (6d6) fire damage on a failed save, or half as much damage on a successful one.","Shadowflame is a fearsome dragon with scales as dark as night, streaked with fiery red veins. Its eyes glow with a sinister light, and its breath weapon spews forth searing flames that can melt stone. The dragon's wings are tattered but powerful, allowing it to soar high above its territory and rain fire upon those who dare to challenge it.","When Elara reaches the grand hall of Blackstone Keep, she finds Shadowflame lying in wait. The dragon immediately attacks, unleashing its fiery breath and using its formidable physical attacks to try and overwhelm her. The battle is fierce and deadly, with Elara using her enchanted sword to finally bring down the beast and end its reign of terror."


### Plot
Need to figure out whether this is relevant, or if it can be merged with Logs table

In [26]:
query = '''
ALTER TABLE PLOT
RENAME COLUMN "index" TO Plot_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [27]:
query = '''
ALTER TABLE PLOT
ADD COLUMN Campaign_ID INTEGER;
'''
run_query(db, query)

OperationalError: duplicate column name: Campaign_ID

In [28]:
query = '''
SELECT * FROM PLOT
'''
run_query(db, query)

,Plot_ID,Setting,Description,Events,Campaign_ID
0,0,The Dragon's Flagon (Tavern),The Dragon's Flagon is a lively tavern with a warm welcoming atmosphere. The walls are adorned with trophies from past adventurers and a large fireplace dominates one side of the room.,"Elara listens to stories and rumors from the locals. An old traveler tells her about an enchanted sword hidden in the Whispering Woods said to be the key to defeating a monster terrorizing the region. Inspired, Elara gathers her gear and sets off on her quest.",None
1,1,Whispering Woods (Wilderness),Whispering Woods is a foreboding forest with a canopy so thick it blocks out most of the sunlight. The air is filled with the sounds of unseen creatures and the ground is covered with a thick layer of leaves.,"Elara encounters several non-combat obstacles including treacherous terrain, ancient traps, and puzzles that test her intellect and resolve. With her determination and skills, she overcomes these challenges and discovers the enchanted sword which is hidden in a small overgrown shrine deep within the woods.",None
2,2,Blackstone Keep (Castle),Blackstone Keep is a crumbling fortress with tall dark towers and walls covered in ivy. Inside it is dark and cold with the air thick with the smell of decay.,"Elara enters the keep navigating its eerie halls and chambers all of which are eerily quiet. She finally reaches the grand hall where she confronts the monster: a fearsome dragon named Shadowflame. Using the enchanted sword, she engages in an epic battle with the dragon. After a fierce and climactic combat, Elara successfully slays the dragon, lifting the curse over the region and bringing peace to the land.",None


### Logs
Main campaign summary table to track story events or character events.
Character ID will indicate that it is an event that happened, if column value is null then it is overall campaign story event

In [29]:
query = '''
ALTER TABLE LOGS
RENAME COLUMN "index" TO Log_ID;
'''
run_query(db, query)

OperationalError: no such column: ""index""

In [30]:
query = '''
ALTER TABLE LOGS
ADD COLUMN Campaign_ID INTEGER;
'''
run_query(db, query)

OperationalError: duplicate column name: Campaign_ID

In [31]:
query = '''
ALTER TABLE LOGS
ADD COLUMN Character_ID INTEGER;
'''
run_query(db, query)

OperationalError: duplicate column name: Character_ID

In [32]:
query = '''
SELECT * FROM LOGS
'''
run_query(db, query)

,Log_ID,Time,Event,X_Coordinate,Y_Coordinate,Campaign_ID,Character_ID


## Add Tracking Tables
- player current inventory

In [33]:
# player inventory tracker
query = '''CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY (
    Campaign_ID INTEGER NOT NULL,
	Character_ID INTEGER NOT NULL,
   	Item_ID INTEGER NOT NULL,
	Quantity FLOAT DEFAULT 0
);
'''

run_query(db, query)

In [34]:
query = '''
SELECT * FROM CHARACTER_INVENTORY
'''
run_query(db, query)

,Campaign_ID,Character_ID,Item_ID,Quantity
